In [3]:
!pip install tensorflow
!pip install scikit-learn
!pip install statsmodels


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/cf/24/271e77c22724f370c24c705f394b8035b4d27e4c2c6339f3f45ab9b8258e/tensorflow-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.18.0 from https://files.pythonhosted.org/packages/76/ad/fa6c508a15ff79cb5409294c293388e0999b7d480f84b65e4287277434fe/tensorflow_intel-2.18.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=24.3.25 from https://files.pythonhosted.org/packages/0e/e2/b066e6e02d67bf526

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.0.2 which is incompatible.
scipy 1.11.1 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.0.2 which is incompatible.


  Obtaining dependency information for numpy>=1.17.3 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     ------------- -------------------------- 20.5/61.0 kB ? eta -:--:--
     ------------- -------------------------- 20.5/61.0 kB ? eta -:--:--
     -------------------------------------- 61.0/61.0 kB 651.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.2/15.8 MB 3.5 MB/s eta 0:00:05
   ---------------------------------------- 0.2/15.8 MB 3.5 MB/s eta 0:00:05
    --------------------------------------- 0.3/15.8 MB 2.2 MB/s eta 0:00:08
    --------------------------------------- 0.3/15.8 MB 2.0 MB/s eta 0:00:08
   - ------------------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 1.26.4 which is incompatible.


In [ ]:
#import libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta


import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Reading the CSV file containing the currency data into a pandas DataFrame
df = pd.read_csv(r'C:\Users\engin\Downloads\all_currencies.csv')

# Dropping unnecessary columns from the DataFrame that won't be used in the analysis
df.drop(['Unnamed: 0', 'Volume', 'Market Cap'], axis=1, inplace=True)

# Resetting the DataFrame's index to a simple range starting from 0 to the length of the DataFrame
df.index = list(range(len(df)))

df.head()

In [4]:
# Filtering the DataFrame for Bitcoin (BTC), Ethereum (ETH), and Bitcoin Cash (BCH) based on the 'Symbol' column
BTC = df[df['Symbol'] == 'BTC']
ETH = df[df['Symbol'] == 'ETH']
BCH = df[df['Symbol'] == 'BCH']

# Converting the 'Date' column for each cryptocurrency to datetime format for proper time-based plotting
BTC['Date'] = pd.to_datetime(BTC['Date'])
ETH['Date'] = pd.to_datetime(ETH['Date'])
BCH['Date'] = pd.to_datetime(BCH['Date'])

# Creating a plot with a figure size of 10x6
plt.figure(figsize=(10, 6))

# Plotting the closing prices for Bitcoin (BTC), Ethereum (ETH), and Bitcoin Cash (BCH)
# Using different colors for each cryptocurrency and adding labels for the legend
plt.plot(BTC['Date'], BTC['Close'], color='orange', label='BTC')
plt.plot(ETH['Date'], ETH['Close'], color='blue', label='ETH')
plt.plot(BCH['Date'], BCH['Close'], color='green', label='BCH')

# Adding a title to the plot
plt.title('Cryptocurrency Closing Prices Over Time')
# Adding a label to the y-axis for clarity
plt.ylabel('Close Price (in $)')
# Displaying a legend to differentiate between the cryptocurrencies
plt.legend()
# Adding grid lines to the plot for better visibility of values
plt.grid(True)
# Displaying the plot
plt.show()

NameError: name 'df' is not defined

In [ ]:
# calculate the correlation matrix between the 'Open', 'High', 'Low', and 'Close' columns in the BTC DataFrame
corr_matrix = BTC[['Open', 'High', 'Low', 'Close']].corr()

# Creating a figure for the heatmap with a size of 8x6
plt.figure(figsize=(8, 6))

# Creating a heatmap of the correlation matrix using seaborn's heatmap function
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt='.2f', linewidths=0.5)

# Adding a title to the heatmap
plt.title('Correlation Matrix (Open, High, Low, Close)')

# Displaying the heatmap
plt.show()


In [ ]:
# Plotting the distribution of prices (Open, High, Low, Close) for BTC
plt.figure(figsize=(12, 8))

# Plotting the histogram and Kernel Density Estimate (KDE) for 'Open' prices in orange
sns.histplot(BTC['Open'], kde=True, color='orange', label='Open', bins=10)

# Plotting the histogram and KDE for 'High' prices in green
sns.histplot(BTC['High'], kde=True, color='green', label='High', bins=10)

# Plotting the histogram and KDE for 'Low' prices in red
sns.histplot(BTC['Low'], kde=True, color='red', label='Low', bins=10)

# Plotting the histogram and KDE for 'Close' prices in blue
sns.histplot(BTC['Close'], kde=True, color='blue', label='Close', bins=10)

# Adding a title to the plot
plt.title('Price Distribution (Open, High, Low, Close)')

# Labeling the x-axis as 'Price' and the y-axis as 'Frequency'
plt.xlabel('Price')
plt.ylabel('Frequency')

# Displaying the legend to identify the price types (Open, High, Low, Close)
plt.legend()

# Adjusting the layout to avoid overlapping labels and make the plot neat
plt.tight_layout()

# Displaying the plot
plt.show()

In [ ]:
# Calculating the 20-day rolling mean and rolling standard deviation for the 'Close' price in the BTC DataFrame
BTC['Rolling_Mean_Close'] = BTC['Close'].rolling(window=20).mean()  # 20-day rolling mean

# The rolling standard deviation calculates the volatility over a 20-day window
BTC['Rolling_Std_Close'] = BTC['Close'].rolling(window=20).std()  # 20-day rolling std dev

# Creating a plot with figure size of 10x6 for a clearer visual presentation
plt.figure(figsize=(10, 6))

# Plotting the 'Close' price using a blue line to show the actual closing prices over time
sns.lineplot(data=BTC, x='Date', y='Close', label='Close Price', color='blue')

# Plotting the rolling mean using a red line to show the smoothed 20-day average of the closing price
sns.lineplot(data=BTC, x='Date', y='Rolling_Mean_Close', label='Rolling Mean (20 Days)', color='red')

# Plotting the rolling standard deviation using a green line to visualize the 20-day price volatility
sns.lineplot(data=BTC, x='Date', y='Rolling_Std_Close', label='Rolling Std Dev (20 Days)', color='green')

# Adding a title to the plot to explain the content being visualized
plt.title('Price, Rolling Mean, and Volatility (20-day window)')

# Labeling the x-axis and y-axis
plt.xlabel('Date')
plt.ylabel('Price / Volatility')

# Rotating the x-axis labels for better readability
plt.xticks(rotation=45)

# Displaying a legend to label the different lines (Close Price, Rolling Mean, Rolling Std Dev)
plt.legend()

# Adjusting the layout to prevent any elements (like labels) from overlapping
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
# Creating a boxplot to visualize the price distribution and outliers for 'Open', 'High', 'Low', and 'Close' columns
plt.figure(figsize=(10, 6))

# Creating a boxplot for the specified columns ('Open', 'High', 'Low', 'Close') in the BTC DataFrame
sns.boxplot(data=BTC[['Open', 'High', 'Low', 'Close']])

# Adding a title to the plot
plt.title('Price Distribution and Outliers (Open, High, Low, Close)')

# Labeling the y-axis as 'Price' to indicate the unit of the values
plt.ylabel('Price')

# Adjusting the layout to prevent any overlapping elements (like labels and title)
plt.tight_layout()

# Displaying the plot
plt.show()


In [ ]:
# Create an IsolationForest model to detect outliers
# 'contamination=0.01' indicates that 1% of the data is expected to be outliers
# The model will label outliers as -1 and inliers as 1
iso_forest = IsolationForest(contamination=0.01)

# Fitting the IsolationForest model to the 'Open', 'High', 'Low', and 'Close' columns of the BTC DataFrame
# The output of fit_predict() is a list of 1s (inliers) and -1s (outliers)
outliers = iso_forest.fit_predict(BTC[['Open', 'High', 'Low', 'Close']])

# Add the outlier labels to the original BTC DataFrame
# 'Outlier' column will store 1 for normal points and -1 for detected outliers
BTC['Outlier'] = outliers

# Plotting the detected outliers
# The plot will show normal points as blue and outliers as red
plt.figure(figsize=(10, 6))

# Create a scatter plot with 'Open' on the x-axis and 'Close' on the y-axis
# We use 'Outlier' as the hue to color the points: normal points (1) are blue, and outliers (-1) are red
sns.scatterplot(data=BTC, x='Open', y='Close', hue='Outlier', palette={1: 'blue', -1: 'red'}, s=100, edgecolor='black')

# Adding a title and labels to the plot
plt.title('Outlier Detection using Isolation Forest (Open vs Close)')
plt.xlabel('Open Price')
plt.ylabel('Close Price')

# Displaying the legend to indicate which color corresponds to normal points and which to outliers
plt.legend(title="Outlier", loc="upper left", labels=['Normal', 'Outlier'])

# Adding grid lines for better readability of the plot
plt.grid(True)

# Adjusting the layout of the plot to prevent overlapping elements
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Plotting the Time Series and Highlighting Outliers
plt.figure(figsize=(10, 6))

# Plot for Open, Close, High, and Low prices
sns.lineplot(data=BTC, x='Date', y='Open', label='Open', color='orange')
sns.lineplot(data=BTC, x='Date', y='Close', label='Close', color='blue')
sns.lineplot(data=BTC, x='Date', y='High', label='High', color='green')
sns.lineplot(data=BTC, x='Date', y='Low', label='Low', color='red')

# Highlight Outliers (those where Outlier column is -1)
outlier_df = BTC[BTC['Outlier'] == -1]
plt.scatter(outlier_df['Date'], outlier_df['Close'], color='yellow', label='Outliers', zorder=5)

# Title and labels
plt.title('Time Series of Prices with Outliers Highlighted (BTC)')
plt.xlabel('Date')
plt.ylabel('Price')

# Show the legend and grid
plt.legend()
plt.grid(True)

# Rotate x-ticks for readability
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Plotting histograms for 'Open', 'High', 'Low', and 'Close' columns in the BTC DataFrame
BTC[['Open', 'High', 'Low', 'Close']].hist(bins=30, figsize=(12, 8))

# Adding a title for the entire figure (all histograms) with font size 16
plt.suptitle("Histograms of Open, High, Low, Close", fontsize=16)

# Customizing the grid line color of the histograms for each subplot
for ax in plt.gcf().get_axes():
    ax.grid(True, color='lightgrey')  # Setting the grid color to light grey

# Display the plot
plt.show()


In [ ]:
# Resampling the data to get monthly samples (last closing price of each month)
monthly_sampling = BTC[['Date', 'Close']].set_index('Date').resample('M').last().reset_index()

# Calculating the 5-period (approximately 30 days) moving average of the 'Close' price
monthly_sampling['5_months_ma'] = monthly_sampling['Close'].rolling(window=5).mean()

# Calculating the 9-period (approximately 100 days) moving average of the 'Close' price
monthly_sampling['9_months_ma'] = monthly_sampling['Close'].rolling(window=9).mean()

# Creating a plot with a figure size of 14x8 for a large and clear visualization
plt.figure(figsize=(14, 8))

# Plotting the 'Close' price as a blue line
plt.plot(monthly_sampling.index, monthly_sampling['Close'], label="Close Price", color='blue')

# Plotting the 30-day moving average as an orange dashed line
# This line represents the 30-day moving average, which smooths out the short-term fluctuations
plt.plot(monthly_sampling.index, monthly_sampling['5_months_ma'], label="5-Months Moving Average", color='orange', linestyle='--')

# Plotting the 100-day moving average as a green dashed line
plt.plot(monthly_sampling.index, monthly_sampling['9_months_ma'], label="9-Months Moving Average", color='green', linestyle='--')

# Adding a title to the plot to indicate what is being visualized
plt.title('Close Price with Moving Averages', fontsize=16)

# Labeling the x-axis as 'Date' and the y-axis as 'Price'
plt.xlabel('Date')
plt.ylabel('Price')

# Displaying a legend to label the different lines (Close Price, 30-Day MA, 100-Day MA)
plt.legend()

# Enabling grid lines to make the plot easier to interpret
plt.grid(True)

# Displaying the plot
plt.show()


## **SARIMAX**

In [ ]:
df = BTC[BTC['Date'] >= '2017-01-01'][['Date', 'Close']].set_index('Date').resample('W').last()
df.head()

In [ ]:
# Test Stationarity using Augmented Dickey-Fuller test
def test_stationarity(timeseries):
  result = adfuller(timeseries)
  print(f"ADF Statistic: {result[0]}")
  print(f"p-value: {result[1]}")
  if result[1] <= 0.05:
      print("The data is stationary.")
  else:
      print("The data is not stationary. Consider differencing or transforming the data.")

test_stationarity(df['Close'])

In [ ]:
#apply differencing
df['diff'] = df['Close'].diff().dropna()

# Split the dataset into training and test sets
train_size = int(len(df) * 0.9)
train, test = df.iloc[:train_size], df.iloc[train_size:]

print(f"Train-Size:{train.shape[0]}\nTest-Size:{test.shape[0]}")

In [ ]:
# SARIMAX Model fitting - Searching for the best model parameters
best_aic = float('inf')
best_order = None
best_seasonal_order = None

# Grid Search for SARIMAX parameters (p, d, q) and seasonal orders (P, D, Q, s)
for p in range(0, 3):
    for d in range(0, 2):
        for q in range(0, 3):
            for P in range(0, 2):
                for D in range(0, 2):
                    for Q in range(0, 2):
                        try:
                            model = SARIMAX(train['diff'], order=(p, d, q), seasonal_order=(P, D, Q, 12))
                            results = model.fit(disp=False)
                            if results.aic < best_aic:
                                best_aic = results.aic
                                best_order = (p, d, q)
                                best_seasonal_order = (P, D, Q, 12)
                        except:
                            continue

print(f"Best SARIMAX model parameters: order={best_order}, seasonal_order={best_seasonal_order}")

# Fit the best SARIMAX model
sarimax_model = SARIMAX(train['diff'], order=best_order, seasonal_order=best_seasonal_order)
results_sarimax = sarimax_model.fit(disp=False)

# Forecasting
forecast_diff = results_sarimax.predict(start=test.index[0], end=test.index[-1], dynamic=False)

# Reverting the differencing: Adding the last known actual value back to the forecasted differenced values
forecast = forecast_diff.cumsum() + train['Close'].iloc[-1]

# Ensure forecast values are clipped to be positive if the actual values are positive
forecast = forecast.clip(lower=0)

# Calculate MAPE, MSE, MAE
mape = mean_absolute_percentage_error(test['Close'], forecast)
mse = mean_squared_error(test['Close'], forecast)
mae = mean_absolute_error(test['Close'], forecast)

print(f"MAPE: {mape:.2f}")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")

In [ ]:
# Plot the actual, predicted, and historical data
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Close'], label='Historical Data', color='blue')
plt.plot(test.index, forecast, label='Predictions', color='red')
plt.plot(train.index, train['Close'], label='Train Data', color='green')
plt.title('SARIMAX Forecasting')
plt.xlabel('Date')
plt.ylabel('Close')
plt.legend()
plt.show()

## **LSTM**

In [ ]:
# Prepare the Data (Scaling)
scaler = MinMaxScaler(feature_range=(0, 1))

# Normalize the data
df_scaled = scaler.fit_transform(df[['Close']])

#Create Sequences for the model
def create_sequences(data, time_steps=1):
    X = []
    y = []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 4  # Use last 4 weeks to predict the next week's value

X, y = create_sequences(df_scaled, time_steps)

# Split the data into train and test sets
train_size = len(df) - 25
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Train-Size:{X_train.shape[0]}\nTest-Size:{X_test.shape[0]}")

In [ ]:
# Reshape the data to fit LSTM/GRU input format (samples, time_steps, features)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=False, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(units=1))  # Output layer

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the Model
history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test), verbose=1)

In [ ]:
# Plotting the training loss and validation loss
plt.figure(figsize=(10, 6))

plt.plot(history.history['loss'], label='Training Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')

# Adding title and labels to the plot
plt.title('LSTM Model Loss vs Validation Loss', fontsize=16)
plt.xlabel('Epochs')
plt.ylabel('Loss')

# Displaying the legend to distinguish between training and validation loss
plt.legend()

# Displaying the grid for better readability
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Forecasting
forecast_scaled = model.predict(X_test)

# Inverse the scaling to get back to the original scale
forecast = scaler.inverse_transform(forecast_scaled)

# Inverse scaling for the true values
y_test_inverse = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate MAPE, MSE, MAE
mape = mean_absolute_percentage_error(y_test_inverse, forecast)
mse = mean_squared_error(y_test_inverse, forecast)
mae = mean_absolute_error(y_test_inverse, forecast)

print(f"MAPE: {mape:.2f}")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))

# Plot the full historical data (train + test)
plt.plot(df.index, df['Close'], label='Historical Data', color='blue')

# Plot the predictions for the test set
plt.plot(df.index[-len(forecast):], forecast, label='Predictions', color='red')

plt.title('LSTM Forecasting')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()

In [2]:
end_date = "2018-11-15"

# Forecasting
forecast_diff = results_sarimax.predict(start=test.index[0], end=datetime.strptime(end_date, "%Y-%m-%d"), dynamic=False)

# Reverting the differencing: Adding the last known actual value back to the forecasted differenced values
forecast = forecast_diff.cumsum() + train['Close'].iloc[-1]

# Ensure forecast values are clipped to be positive if the actual values are positive
forecast = forecast.clip(lower=0)

def get_weekly_dates(start_date_str, end_date_str, forecast):
    # Convert the input date strings to datetime objects
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    # Initialize an empty list to store the dates
    dates = []

    # Generate dates at weekly intervals
    current_date = start_date
    while len(dates) != len(forecast):
        dates.append(current_date)  # Convert to string for output
        current_date += timedelta(weeks=1)
    return dates
start_date = test.index[0]
end_date = end_date

weekly_dates = get_weekly_dates(start_date, end_date, forecast)

NameError: name 'results_sarimax' is not defined

In [3]:
# Plot the actual, predicted, and historical data
plt.figure(figsize=(10, 6))
plt.plot(df.index, df['Close'], label='Historical Data', color='blue')
plt.plot(weekly_dates, forecast, label='Predictions', color='red')
plt.title('SARIMAX Forecasting')
plt.xlabel('Date')
plt.ylabel('Close')
plt.legend()
plt.show()

NameError: name 'df' is not defined

<Figure size 1000x600 with 0 Axes>